<a href="https://colab.research.google.com/github/caropilardiaz/Mentoria-Imagenes_Drones/blob/master/2_PracticoAyC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Diplomatura en Ciencias de Datos, Aprendizaje Automático y sus Aplicaciones**

##**Mentoria Diplomatura en Ciencia de Datos 2020**
### "Detección del desarrollo del cultivo con imágenes de Drones"


###**Analisis y Curación de datos**
Se propone la elaboración de un informe o presentacion, en formato jupyter notebook que responda a las cuestiones solicitadas en cada seccion de este notebook.El informe debe contener los resultado obtenidos de cada items.



En este práctico partiremos de los datos crudos "Dron1.csv" como al inicio del práctico de visualización, e incluiremos algunos más. La idea general es dejar preparado un único dataset que nos permita alimentar futuros modelos de aprendizaje automático.

Deberán incluir la información meteorológica "clima_2018.csv" pertinente en el dataset final.Para unificar los reportes, se recomienda utilizar el método "merge" de pandas ó el que ustedes elijan.

Es importante que cada decisión que tomen sobre los datos quede debidamente documentada.

Se recomienda trabajar empleando algún checklist para asegurarse de cumplir todas las etapas del proceso de curación. Pueden utilizar el checklist que vieron en clase, o algún otro que consideren más apropiado (en las referencias tienen uno a modo de ejemplo).

###ACLARACIONES###



####**Chequeos puntuales por campo**
Campo **"RDTO"**:Es una variable contínua que representa uno de los targets para algoritmos con salidas de variables contínuas

Campo **"Conj de datos"**: Verán que en este campo aparecen dos categorias que se asocian al momento de siembra diferente dentro del mismo ensayo. Las categorías "1rasiembra", "2dasiembra" indican agronómicamente siembra temprana y siembra tardía no puede haber un tipo distinto de categoría.

Campo **"ciclo"**: Este campo también indica categorías bipuntuales.Cada genotipo tiene un determinado ciclo fenológico, ésta variable indica a que ciclo pertenece cada uno.Son grupos de interés y se encuentran tanto en fechas tempranas como en tardías. 

Campo **"Roya"**: Noten que este campo en particular contiene muchos valores pertencientes a la categoría nulos que significa ausencia de la enfermedad. Deberán determinar que hacer si es necesario incorporarla o no.

Campo **"Manchas Foliares"**: No presenta variabilidad ni valores contundentes, significa que el cultivo en general no adolece de la enfermedad. Deberán decidir y documentar.

Campo **"Fusariosis, Carbón y Adversidad"**:Observa que las categorías son muy diferentes en cardinalidad significa que no presenta enfermedades el cultivo. Decidir y documentar acerca de éstas variables.Acá deberán evaluar si conviene modificar el campo. Por ejemplo, mantener las categorías  y recategorizar las demás como "Otras". En cualquier caso, justifiquen la decisión tomada.

Campo **"Zadok"**; Es nuestro otro target y es una variable categórica la variable zadok describe el estadío fenológico en que se encuentran las variedades e inclusive el ensayo discrimina dos estadíos fenológicos de la misma variedad utilizando fechas de siembras diferentes. Observar y documentar.
 
Campo **"Fecha espigazón"**: Esta es la fecha en la que se medió el comienzo de la espigazón para cada variedad. Corroborar que la misma es anterior a la fecha de madurez.Verificar que las fechas pertenecen al rango temporal correspondiente (en este caso,sólo al año 2018). Son datos cargados manualmente con muchos errores los 2016 7 2015 agregar a 2018, 

Campo **"Fecha madurez"**: Fecha que se mide manualmente en todas la variedades. Datos muy sucios pero que se deben corresponder a fechas posteriores a las fechas de espigazón, durante el 2018.Los 2016 y 2015 agregar a 2018.

Campo **"Variedades"**: Este campo indica los genotipos que se usaron en el ensayo, es interesante observar que las mismas variedades fueron implantadas en una fecha y en la otra. Deberán observar que variedades coincide exactamente con la variable parcela, decidir y documentar acerca del tratamiento de las mismas.

Campos **"Rep, Fila y Col"**: Van a ver que estos campos son bastante irrelevantes. Acá la recomendación es distinguir sólo si pertenece a la variedad o no y en que zadok está la misma variedad,en definitiva tenemos 3 parcelas con la misma variedad en la primera fecha de siembra, y 3 parcelas con exactamente la misma variedad en la segunda fecha de siembra pero en distinto estadío Zadok. 


###**Incorporación de información climática**
El dataset "climaReporte_2018.csv" contiene información climática segmentada por día. Esta información es de carácter público y fue obtenida del Sistema de gestión Clima del Dpto de información agronómica de BCCBA,  https://clima.bccba.org.ar/accounts/login/?next=/ . Determinen qué información puede ser relevante para la problemática de la mentoría e incorporenla al dataset agronómico obteniendo un dataset final.



###Referencias
https://dimewiki.worldbank.org/wiki/Checklist:_Data_Cleaning

##**Importamos paquetes**

In [18]:
# Importación de las librerías necesarias
import numpy as np
import pandas as pd
# Puede que nos sirvan también
import matplotlib as mpl
mpl.get_cachedir()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [19]:
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)
pd.set_option('max_colwidth', 151)

In [20]:
import os

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
DATA_DIRECTORY = '/content/drive/My Drive/Mentoria-Drones-master/data/'

##Consigna para Curación y Exploración del Dataset

##I. Rutina de Curación
Inicialmente, con el objetivo de preparar los datos que alimentarán futuros modelos de aprendizaje automático (ML), se propone seguir la siguiente checklist para la limpieza de los datos de nuestro proyecto. Esta checklist es la misma que utilizaron en el primer práctico de la materia y nos será de utilidad como guía para curar el dataset. A modo de ayuda, en esta notebook encontrarán una especie de template que sigue la checklist y que deberán ir completando.

Cada decisión tomada deberá quedar registrada de manera explícita y clara. Luego de pasar por todos los puntos de la checklist propuesta, deberán almacenar en un nuevo archivo los datos resultantes. A los fines de realizar este práctico, se utilizará el dataset original, pero descartando todas aquellas columnas que se hayan calculado en base a features preexistentes, ya que éstas están relacionadas a decisiones que adoptaremos más adelante, como por ejemplo, sobre si es necesario crear nuevas features y si incluirlas o no. Recuerden que la ciencia de datos es un proceso circular, continuo y no lineal. Es decir, si los datos requieren de mayor procesamiento para satisfacer las necesidades de algoritmos de ML (cualesquiera de ellos), vamos a volver a la etapa inicial para, por ejemplo, crear nuevas features, tomar decisiones diferentes sobre valores faltantes o valores atípicos (outliers), descartar features, entre otras.
###**1. Importación de Datos**
####1.1 Cargamos datos agronómicos

In [23]:
#Editar con el path correspondiente a sus datos
dfa = pd.read_csv(os.path.join(DATA_DIRECTORY,'Dron1.csv'))



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
print(dfa.shape)
dfa.sample(5)

(38920, 45)


,Longitude,Latitude,Conjunto de datos,ID,Ciclos,Variedad,Rep,Parcela,ZADOCK,fila,columna,Azul,Verde,Rojo,Borde rojo,Infra Rojo,NDVI,NDRE,SAVI,SCCI,GNDVI,GVI,RVI,RedEdgeVI,LNVI,GRVI,MGRVI,NGRVI,Roya (%),Manchas Foliares,Fusariosis,Carbon (%),Adversidades,Fecha de espigaz?,Fecha de madurez,dias entre fechas,Vuelco,Altura,Aspecto,Peso hecto,Peso de 1000 granos,p grano,hum,PAJUST,RDTO
31446,-2.057.290.483.445.820,-2.120.812.910.021.590,2daSiembra,11104,CC,Variedad_T,1,115,6.3,1,34,52700,11790,49000,29710,13022,-0.5801,-0.3905,-0.8701,0.6732,0.0497,1.1045,0.2658,-0.8248,10.2992,-0.6121,-0.8905,-1.1229,0,diez,cero,0,0.00,27/9/2018,20/11/2018,54,2.0,80.0,2.5,76.1,37.0,3825,9.8,4012,8024
21945,-2.057.278.477.116.630,-2.120.800.818.655.500,2daSiembra,1603,CL,Variedad_N,1,79,4.7,1,44,44400,95700,40000,25600,13194,-0.5039,-0.3198,-0.7559,0.6346,-0.7577,0.1379,0.3299,-0.6628,10.1503,0.4105,0.7026,1.4234,1,cero,cero,0,0.00,7/10/2018,2/12/2018,56,1.0,75.0,2.0,70.4,28.0,2470,7.9,2645,5290
16152,-2.057.305.208.509.710,-2.120.842.096.941.140,1er Siembra,7997,CC,Variedad_T,2,57,6.6,2,3,60700,13750,59700,31720,12001,-0.6652,-0.4510,-0.9979,0.6780,-0.0679,0.8728,0.2010,-0.9720,10.3647,-0.6256,-0.8993,-1.1120,0,cero,uno/uno,0,0.00,14/9/2015,19/11/2015,66,1.0,70.0,3.5,79.6,39.0,2175,10.3,2269,4537
13024,-2.057.314.170.535.450,-2.120.845.636.074.510,1er Siembra,4853,CC,Variedad_O,1,44,6.1,1,11,45000,10180,43900,23590,10177,-0.6236,-0.3972,-0.9354,0.6370,-0.0001,0.9997,0.2318,-0.8407,10.0686,-0.6235,-0.8979,-1.1137,0,cero,uno/uno,0,0.05,16/9/2018,28/11/2018,73,1.0,80.0,2.0,76.5,39.0,2110,10.5,2196,4392
2761,-2.057.304.645.649.740,-2.120.834.826.734.160,1er Siembra,14931,CL,Variedad_F,1,10,6.5,1,23,58600,14860,58500,42430,19912,-0.4921,-0.3612,-0.7382,0.7340,0.1453,1.3400,0.3404,-0.7565,10.6556,-0.5949,-0.8788,-1.1380,3,cero,cero,0,0.05,19/9/2018,23/11/2018,65,1.0,87.0,3.0,73.0,37.0,2715,9.6,2854,5708


####1.2.Cargamos dataset de información climática


In [25]:
#Editar con el path correspondiente a sus datos
dfc = pd.read_csv(os.path.join(DATA_DIRECTORY,'ClimaReporte_2018.csv'))



In [26]:
print(dfc.shape)
dfc.sample(5)

(122, 20)


,Fecha,Tem. [°C] Prom,Tem. [°C] Mín,Tem. [°C] Máx,Presion [hPa] Prom,Presion [hPa] Mín,Presion [hPa] Máx,Temp suelo [°C] Prom,Temp suelo [°C] Mín,Temp suelo [°C] Máx,Lluvia Acum[mm],Máx Inten. de Lluvia [mm/10min],Inten de Lluvia [mm/h],Vel de Viento [Km/h] Prom,Vel de Viento [Km/h] Máx,Hum [%HR] Prom,Hum [%HR] Mín,Hum [%HR] Máx,Raf. de Viento [km/h Prom,Raf. de Viento [km/h Máx
95,05/12/2018,21.720,17.22,28.53,977.873,970.625,984.375,-14.262,-15.3,-13.2,0.0,0.0,0.0,10.243,22.249,53.015,34.12,79.69,17.556,42.728
0,01/09/2018,7.607,2.66,13.09,988.546,985.000,991.250,13.056,12.2,14.2,0.2,0.2,1.2,13.228,27.278,64.772,47.11,79.58,21.625,61.376
78,18/11/2018,21.381,13.61,28.16,981.584,978.125,986.250,-23.967,-24.3,-23.6,0.0,0.0,0.0,12.075,18.849,29.063,11.49,52.80,20.936,30.678
66,06/11/2018,20.482,18.36,23.47,983.112,977.500,988.125,-14.749,-15.3,-14.0,16.2,3.2,19.2,9.023,29.491,90.301,77.68,98.46,15.454,49.004
100,10/12/2018,28.052,18.68,36.39,974.670,968.125,979.375,-13.749,-17.0,-12.0,0.0,0.0,0.0,17.126,36.110,51.795,28.76,82.77,29.747,64.233


####Tipos de datos que tenemos

In [27]:
dfa.dtypes

Longitude               object
Latitude                object
Conjunto de datos       object
ID                       int64
Ciclos                  object
Variedad                object
Rep                      int64
Parcela                  int64
ZADOCK                 float64
fila                     int64
columna                  int64
Azul                     int64
Verde                    int64
Rojo                     int64
Borde rojo               int64
Infra Rojo               int64
NDVI                   float64
NDRE                   float64
SAVI                   float64
SCCI                   float64
GNDVI                  float64
GVI                    float64
RVI                    float64
RedEdgeVI              float64
LNVI                   float64
GRVI                   float64
MGRVI                  float64
NGRVI                  float64
Roya (%)                 int64
Manchas Foliares        object
Fusariosis              object
Carbon (%)               int64
Adversid

In [28]:
dfc.dtypes

Fecha                               object
Tem. [°C] Prom                     float64
Tem. [°C] Mín                      float64
Tem. [°C] Máx                      float64
Presion [hPa] Prom                 float64
Presion [hPa] Mín                  float64
Presion [hPa] Máx                  float64
Temp suelo [°C] Prom               float64
Temp suelo [°C] Mín                float64
Temp suelo [°C] Máx                float64
Lluvia Acum[mm]                    float64
Máx Inten. de Lluvia [mm/10min]    float64
Inten de Lluvia [mm/h]             float64
Vel de Viento [Km/h] Prom          float64
Vel de Viento [Km/h] Máx           float64
Hum [%HR] Prom                     float64
Hum [%HR] Mín                      float64
Hum [%HR] Máx                      float64
Raf. de Viento [km/h Prom          float64
Raf. de Viento [km/h Máx           float64
dtype: object

#### 1.3. Asegurar la Existencia de IDs o Claves Únicas

##### El siguiente paso implica chequear que no existen datos duplicados y que las claves, si existen, son únicas.

####1.4. Despersonalizar Datos si hiciera falta y Guardarlos en un Nuevo Archivo

 Nunca Modificar los Datos Crudos u Originales

####**2. Pasos de Limpieza Necesarios**


####2.1.Etiquetas de Variables/Columnas y Problemas de Codificación/Encoding

In [ ]:
import chardet

In [ ]:
import requests
rawdata = requests.get(dfa)
result = chardet.detect(rawdata.content[:100000])
result

####2.2.Tratamiento de Valores Faltantes


####Para analizar los valores faltantes, primero deberán saber cuántos existen por campo y cuánto representan del total:



In [ ]:
valores_faltantes = pd.DataFrame([dfa.isnull().sum(),
                                   dfa.isnull().sum()/len(dfa)]).transpose().rename(
    columns = {0:'Cantidad_NaN',1:'Porcentaje_Nan_s_Total'})

valores_faltantes.loc[valores_faltantes['Cantidad_NaN']>0].style.format({'Porcentaje_Nan_s_Total':"{:.2%}"})

,Cantidad_NaN,Porcentaje_Nan_s_Total


####2.3.Codificación de Variables Categóricas

En el dataset agronómico, exiten categorías de enfermedades que se podrían cambiar la cardinalidad. Agrupando varios niveles y dejando el nivel cero.

####2.4. No Cambiar los Nombres de las Variables de la Fuente de Origen


####2.5. Verificación de Consistencia de Datos

#####Este es el paso más analítico, en donde se deben aplicar reglas de integridad.



#### Template para tests de integridad

In [ ]:
def test1(df):
    """
    Devuelve 0 si el test es exitoso, y 1 si el test falla
    """
    
    return 1

def test2(df):
    """
    Devuelve 0 si el test es exitoso, y 1 si el test falla
    """
    return 0

def test_integridad(df):
    """
    Ejecuta uno por uno los tests e informa el resultado
    """
        
    print('Test 1: %s' % ('ERROR' if test1(df) else 'EXITOSO'))
    print('Test 2: %s' % ('ERROR' if test2(df) else 'EXITOSO'))
        
    return

test_integridad(1)

Test 1: ERROR
Test 2: EXITOSO


####2.6. Identificar y Documentar Valores Atípicos/Outliers

####2.7. Guardar el Set de Datos con un Nombre Informativo

####**3.Chequeos por campo**

####3.1. Ordenar Variables/Columnas

####3.2. Quitar Variables/Columnas Irrelevantes
######Este paso es muy importante. Deben decidir con cuál de las targets van a modelar, ó con las dos (Rdto, ZADOK)es opcional. LAS variables que tienen muy alta correlación CON RDTO como peso ajustado, humedad no tienen sentido retenerlas. Son variables que conforman la target rdto. Evitar eliminar las que tengan que ver con los índices de vegetación.Recomiendo aplicar mapa de correlación.

####3.3. Renombrar Variables
Algunos names son muy largos

####3.4. Categorizar Variables que Contengan “Otros”
#####Sobretodo la variables de enfermedades, como Carbón, fusariosis,etc, presentan desbalance en los niveles de sus categorías. Decidir si nombrar otros y agrupar a las que contengan menos entrada

####3.5. Agregar Metadata a los Datos
#####Cuando y como fueron obtenidos, limpieza realizada, decisiones implementadas, asunciones, etc.




#II.Incorporación de datos climáticos

####**1. Incorporación de información climática**

##### Variables climáticas a utilizar para incorporar al dataset. Explorar

####**2. Unificar**

#####Unificar el dataset obtenido (luego de aplicar la limpieza) con los datos climáticos.